In [100]:
import pandas as pd
import datetime
from datetime import timedelta
from datetime import date
import openpyxl
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font
from pathlib import Path
from collections import Counter

print("загрузка данных из таблицы")

#3 года
arg1 = datetime.datetime.today()
timeD=arg1-timedelta(1095)

df=pd.read_excel(r'femida.xlsx')
df["sccStatus"]=np.where((df["Дата подачи ЗОП"]!="-"), df["sccStatus"], np.nan) 
df=df[df["sccStatus"].notnull()]

df=df[(df["sccStatus"]=="В работе")
      &(df["Вид события"]=="Мошенничество в корпоративном кредитовании")
      &(df["Дата подачи ЗОП"]>=timeD)]

del df["Наименование ЮЛ"]
del df["ИНН ЮЛ"]
del df["Дата создания карточки"]
del df["Состояние рассмотрения"]
del df["Ущерб причинен"]
del df["Ущерб оценивается"]
del df["Ущерб предотвращенный"]
del df["ТБ/ЦА"]
del df["Исполнитель"]
del df["sccStatus"]
del df["Приоритет"]
del df["Регион"]
del df["Населенный пункт"]
del df["Тип источника"]
del df["Вид события"]
del df["Механизм реализации"]
del df["Дата возбуждения УД"]
del df["Номер УД"]
del df["Дата предъявления обвинения"]
del df["Дата передачи дела в суд"]
del df["Тип ПО"]
del df["Фигуранты"]
del df["Соучастники"]
del df["Тип потерпевшего"]
del df["Потерпевшие"]
del df["Подозреваемые"]
del df["Потерпевший СБЕР"]
del df["Актуальное состояние ХР ЗОП"]
del df["Дата передачи дела в суд первой инстанции"]
del df["Дата события"]

df.loc[df["Ущерб возмещенный"]=="-", "Ущерб возмещенный" ]=0
df.loc[df["Ущерб возмещенный"]=="0.00", "Ущерб возмещенный" ]=0
df['Ущерб возмещенный'] = df['Ущерб возмещенный'].astype(float)

df.loc[df["Сумма ущерба"]=="0.00", "Сумма ущерба" ]=0
df.loc[df["Сумма ущерба"]=="-", "Сумма ущерба" ]='0'
df['Сумма ущерба'] = df['Сумма ущерба'].astype(float)

df["Подразделение"]=df["Подразделение"].str.replace('-', 'Аппарат ТБ')

df["Ззаяв"]=1
df["Итог Кву"]=""

df.to_excel(r'Итог карточки.xlsx', index=False)

print("загрузка шаблона")
wb=openpyxl.load_workbook("Итог карточки.xlsx")
ws=wb.active
rows = ws.max_row

print("запуск перебора строк")
for i in range(2, rows+1):
    if (ws.cell(row=i, column=2).value)!=0 and ws.cell(row=i, column=6).value!=0:
        ws.cell(row=i, column=7).value =float(float(float(ws.cell(row=i, column=3).value)/(ws.cell(row=i, column=2).value)))
    else: ws.cell(row=i, column=7).value =0

ws.column_dimensions['A'].width = 15
ws.column_dimensions['B'].width = 15
ws.column_dimensions['C'].width = 20
ws.column_dimensions['D'].width = 25
ws.column_dimensions['E'].width = 20

wb.save(f'Итог карточки.xlsx')

df=pd.read_excel(r'Итог карточки.xlsx')

wb = openpyxl.load_workbook(r'Итог карточки.xlsx')
ws = wb.active
rows = ws.max_row
print("перебор2")

for i in range(2, rows+1):
    ws.cell(row=rows+1, column=6).value =df.loc[df['Номер карточки']!="", "Ззаяв"].sum()
    ws.cell(row=rows+1, column=7).value =df.loc[df['Номер карточки']!="", "Итог Кву"].sum()
    ws.cell(row=rows+2, column=7).value ="%.0f%%"%(((ws.cell(row=rows+1, column=7).value)/(ws.cell(row=rows+1, column=6).value))*100)
    
wb.save(f'Итог карточки.xlsx')
print("готово")

загрузка данных из таблицы
загрузка шаблона
запуск перебора строк
перебор2
готово
